# Imports

In [1]:
# Import standard libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# API
import requests
import snowflake.connector

# Date
import datetime as dt
import time
from datetime import timedelta

In [2]:
silver = pd.read_csv('Acuity.csv')
acuity = pd.read_csv('acuity_intake_source_2021.csv')
LCM = pd.read_csv('LCM.csv')

# Change dates to datetime Dtype
silver['APPOINTMENT_BODY_DATE'] = pd.to_datetime(silver['APPOINTMENT_BODY_DATE'])
acuity['Date Scheduled'] = pd.to_datetime(acuity['Date Scheduled'])

In [3]:
# drop
na_dict = {'checkoutsuccess' : np.nan,
          'cx' : np.nan,
          'emaildrip' : np.nan,
          'hmafollowup' : np.nan,
          'trackingpage' : np.nan,
          'brightback' : np.nan,
          'socialdm' : np.nan,
          'dr_hma' : np.nan,
          'dr_consult' : np.nan,
          'lcm_210127-newyear-nonsubw-20' : np.nan,
          'lcm_210127-newyear-nonsub-b-15' : np.nan,
          'lcm_welcome_M_D21' : np.nan,
          'lcm_210127-newyear-nonsubw-15' : np.nan}

acuity['Campaign'].replace(na_dict, inplace = True)

acuity.Campaign.value_counts()

hmapage    186
ndpage      94
Name: Campaign, dtype: int64

In [4]:
acuity.drop(columns=['Start Time', 'End Time', 'First Name', 'Last Name', 'Phone',
       'Type', 'Calendar', 'Appointment Price', 'Paid?', 'Amount Paid Online',
       'Certificate Code', 'Notes', 'Label', 'Scheduled By'], inplace=True)

In [5]:
acuity.dropna(inplace = True)

In [6]:
acuity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280 entries, 170 to 1634
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Email           280 non-null    object        
 1   Date Scheduled  280 non-null    datetime64[ns]
 2   Campaign        280 non-null    object        
 3   Appointment ID  280 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 10.9+ KB


In [7]:
# Drop unneeded columns
silver.drop(columns=['EVENT_TS', 'EVENT_TS_UTC', 'ANONYMOUS_ID','APPOINTMENT_BODY_FIRST_NAME',
       'APPOINTMENT_BODY_DATETIME_CREATED', 'APPOINTMENT_BODY_CALENDAR_ID','APPOINTMENT_BODY_CALENDAR_TIMEZONE',\
                    'APPOINTMENT_BODY_TIMEZONE', 'APPOINTMENT_BODY_LAST_NAME',\
                    'APPOINTMENT_BODY_NO_SHOW', \
                    'APPOINTMENT_TYPE_ID'], inplace=True)

# drop irrelevant appointments
na_dict= {'Hair Mineral Analysis - Acquisition':np.nan,
          'Follow Up Consultation':np.nan,
          'Customer consultation':np.nan,
          'Case Study':np.nan}

silver['APPOINTMENT_BODY_TYPE'].replace(na_dict, inplace = True)

clean = silver.groupby('EMAIL').agg({"APPOINTMENT_BODY_DATE":'min',\
                                'APPOINTMENT_BODY_TYPE':'last',\
                                'APPOINTMENT_BODY_CALENDAR':'last','APPOINTMENT_BODY_ID':'min'\
                               })
clean.shape

(4958, 4)

In [8]:
intake = pd.merge(acuity,clean, how='left',left_on='Appointment ID', right_on='APPOINTMENT_BODY_ID',copy=False)
intake.Email.nunique()

277

In [11]:
intake2 = pd.merge(intake,LCM, how='left',left_on='Email', right_on='CUSTOMER_EMAIL',copy=False)
intake2.Email.nunique()

277

In [12]:
# Export data
intake2.to_csv('intake2.csv')
intake2.describe()

,Appointment ID,APPOINTMENT_BODY_ID
count,2.800000e+02,2.510000e+02
mean,5.137600e+08,5.134765e+08
std,6.488579e+06,6.213369e+06
min,5.029057e+08,5.029057e+08
25%,5.080363e+08,5.080143e+08
50%,5.136479e+08,5.134402e+08
75%,5.187554e+08,5.180592e+08
max,5.296502e+08,5.284136e+08


In [13]:
acuity.to_csv('QA.csv')